# Integrazione finale

Integro tra loro i dati sui titoli presi da wikipedia e quelli presi da IMDb, aggiungo a mano le informazioni su qualche titolo mancante.

In [1]:
import pandas as pd

### Leggo i dataset

Dati integrati con scraping da Wikipedia

In [2]:
# Dataset con titoli e dati di wikipedia

film_wiki = pd.read_csv("Dati_integrati/wiki_film_dic-mar.csv", index_col = 0)
serie_tv_wiki = pd.read_csv("Dati_integrati/wiki_serie_dic-mar.csv", index_col = 0)

Dataset con dati presi da IMDb.

In [3]:
# Dataset con dati imdb

imdb = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/ratings_imdb_dic-mar2021.csv", index_col = 0)

Dati da Flixpatrol: classifiche Netflix giorno per giorno.

In [4]:
# Tutti i dati
netflix = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/netflix_dic-mar2021.csv", index_col = 0)

# Le tre classifiche singole
top10 = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_dic-mar2021.csv", index_col= 0)
top_film = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_film_dic-mar2021.csv", index_col= 0)
top_serie = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_serie_dic-mar2021.csv", index_col= 0)

Creo i due nuovi database __movies__ e __tv_series__ in cui incrocio i dati di Wikipedia con i rating di IMDb.

In [5]:
movies = pd.merge(film_wiki, imdb, left_on = 'Titolo', right_on = 'Title')
movies = movies.drop(columns = ['Title', 'Type'])

In [6]:
tv_series = pd.merge(serie_tv_wiki, imdb, left_on = 'Titolo', right_on = 'Title')
tv_series = tv_series.drop(columns = ['Title', 'Type'])

### Estraggo,correggo i titoli e li ordino in ordine alfabetico

Alcuni titoli presentano delle imperfezioni perché contengono un apostrofo.

In [7]:
titoli_raw = list(set(netflix['name']))  # titoli così come sono nel database netflix

titoli = titoli_raw.copy() # titoli che saranno corretti

In [8]:
len(titoli_raw)

208

In tutto ho 208 titoli. Alcuni titoli nel database originario (contenuti in titoli_raw) sono scritti male, quindi li correggo. titoli_raw conterrà i titoli originari, mentre titoli i titoli corretti.

In [9]:
import re

In [10]:
# Stampo i titoli scritti male (= con un apostrofo)

for t in titoli_raw:
    occ = re.compile('^.*&\#39;.*$')
    
    if occ.match(t):
        print(t)

Finding &#39;Ohana
A Dog&#39;s Way Home
I&#39;m Not a Killer
The Queen&#39;s Gambit
Don&#39;t Listen
Dawson&#39;s Creek
So My Grandma&#39;s a Lesbian!
To All the Boys I&#39;ve Loved Before
Ma Rainey&#39;s Black Bottom


In tutto ho 9 titoli problematici.

In [11]:
# Correggo (in titoli) i titoli scritti male

for t in titoli:
    
    occ = re.compile('(^.*)(\&\#39;)(.*$)')
    found = occ.match(t)
    
    if found:
        titoli.remove(t)
        pulita = re.sub(r'(\&\#39;)',"\'" ,t)
        titoli.append(pulita)

In [12]:
# Ordino in ordine alfabetico

titoli = sorted(titoli)
titoli_raw = sorted(titoli_raw)

In [13]:
# Prova

print(titoli_raw[190])
print(titoli[190])

To All the Boys I&#39;ve Loved Before
To All the Boys I've Loved Before


# Creazione dell'indice di popolarità

Per ogni film f, considerando la classifica totale, il suo indice di popolarità è dato da:

$$ I(f) = \frac{\sum_{g \in G}(10 - ranking(f))}{10*N}$$

dove G è l'insieme dei giorni considerati, N è la cardinalità di G, e il ranking(f) vale 0 se f è primo in classifica, 2 se è secondo eccetera, e infine vale 10 se non è in classifica.
Il ranking vale dunque: $$ranking(f) = ranking - 1$$
dove il ranking è la colonna del dataframe in esame.

In [14]:
lung_titoli = len(titoli_raw)

In [15]:
# In tutto ho 121 giorni (31 (dicembre) + 31 (gennaio) + 28 (febbraio) + 31 (marzo))
num_giorni = 121

In [16]:
# Calcolo indice relativo alla classifica generale (contenuta nel df top10)

def indice_popolarità_gen(riga):
    
    # Stringa con il titolo che considero
    t = riga['Titolo']
    
    for i in range(lung_titoli):
        if titoli[i] == t:
            break

    t_raw = titoli_raw[i]

    # Raggruppo il dataframe delle classifiche per il titolo, così ho i dati relativi ai giorni
    # in cui quel titolo è stato in classifica, ottengo una serie (s) con la posizione (ranking) per ogni 
    # giorno in cui quel titolo è stato in classifica
    s = top10[top10['name'] == t_raw][['name','ranking']]
    
    # Calcolo il numeratore dell'indice
    indice = 0

    for r in s['ranking']:
        indice = indice + (10 - (r - 1)) 
    
    # Divido ora il numeratore per denominatore ottenendo l'indice
    return indice/(10*num_giorni)

Uso la stessa funzione anche per calcolare l'indice con una classifica diversa, cambio solo i df da cui prendo i dati.

In [17]:
# Calcolo indice relativo alla classifica dei film (contenuta nel df top_film)

def indice_popolarità_film(riga):
    
    # Stringa con il titolo che considero
    t = riga['Titolo']
    
    for i in range(lung_titoli):
        if titoli[i] == t:
            break

    t_raw = titoli_raw[i]
    
    # Raggruppo il dataframe delle classifiche per il titolo, così ho i dati relativi ai giorni
    # in cui quel titolo è stato in classifica, ottengo una serie (s) con la posizione (ranking) per ogni 
    # giorno in cui quel titolo è stato in classifica
    s = top_film[top_film['name'] == t_raw][['name','ranking']]
    
    # Calcolo il numeratore dell'indice
    indice = 0

    for r in s['ranking']:
        indice = indice + (10 - (r - 1)) 
    
    # Divido ora il numeratore per denominatore ottenendo l'indice
    return indice/(10*num_giorni)

In [18]:
# Calcolo indice relativo alla classifica delle serie (contenuta nel df top_serie)

def indice_popolarità_serie(riga):
    
    # Stringa con il titolo che considero
    t = riga['Titolo']
    
    for i in range(lung_titoli):
        if titoli[i] == t:
            break

    t_raw = titoli_raw[i]
    
    # Raggruppo il dataframe delle classifiche per il titolo, così ho i dati relativi ai giorni
    # in cui quel titolo è stato in classifica, ottengo una serie (s) con la posizione (ranking) per ogni 
    # giorno in cui quel titolo è stato in classifica
    s = top_serie[top_serie['name'] == t_raw][['name','ranking']]
    
    # Calcolo il numeratore dell'indice
    indice = 0

    for r in s['ranking']:
        indice = indice + (10 - (r - 1)) 
    
    # Divido ora il numeratore per denominatore ottenendo l'indice
    return indice/(10*num_giorni)

Nei due dataframe la colonna _Indice_generale_ si riferisce all'indice i popolarità per quel titolo nella classifica generale, invece _Indice_tipo_ di riferisce all'indice di popolarità per la classifica specifica (solo film o solo serie tv).


Applico ora la funzione al dataframe dei film per creare le nuove colonne 'Indice generale' e 'Indice tipo':

In [19]:
# Dataframe dei film

movies['Indice_generale'] = movies.apply(indice_popolarità_gen, axis = 1)
movies['Indice_tipo'] = movies.apply(indice_popolarità_film, axis = 1)

In [20]:
movies.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata,Regista,Budget,Box_office,Sommario,Trama,Cast_testo,Produzione_testo,Critica,Average_rating,Num_votes,Indice_generale,Indice_tipo
0,365 Days,Poland,Ekipa Sp. z o.o. Future Space Next Film TVN,Next Film,Anna-Maria Sieklucka Michele Morrone Bronisław...,Tomasz Klimala Barbara Białowąs Tomasz Mandes ...,114 minutes,Barbara Białowąs Tomasz Mandes,NaN,$9.5 million[1],365 Days (Polish: 365 Dni) is a 2020 Polish er...,Section: Plot (1):\nAfter a meeting between th...,Section: Cast (1):\nMichele Morrone as Don Mas...,Section: Production (1):\nThe film scenes were...,Section: Reception (1):\nThe film made the top...,3.2,54757.0,0.00000,0.000826
1,5 Is the Perfect Number,Italy,Rai Cinema,01 Distribution,Toni ServilloValeria GolinoCarlo BuccirossoIai...,Igort,100 minutes,Igort,NaN,NaN,5 Is the Perfect Number (Italian: 5 è il numer...,"Section: Plot (1):\nIn the 1970s Naples, the r...",Section: Cast (1):\n\nSubsections (0):\n,NaN,NaN,5.3,1075.0,0.00000,0.030579
2,A California Christmas,United States,ESX Entertainment,NaN,Lauren Swickard Josh Swickard,Lauren Swickard,106 minutes,Shaun Piccinino,NaN,NaN,A California Christmas is a 2020 Christmas fil...,Section: Plot (1):\nJoseph Van Aston lives a c...,Section: Cast (1):\n\nSubsections (0):\n,NaN,NaN,5.7,6543.0,0.06281,0.114876


Idem con il dataframe delle serie tv:

In [21]:
# Dataframe delle serie tv

tv_series['Indice_generale'] = tv_series.apply(indice_popolarità_gen, axis = 1)
tv_series['Indice_tipo'] = tv_series.apply(indice_popolarità_serie, axis = 1)

In [22]:
tv_series.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata_episodio,Genere,N_stagioni,N_episodi,Sommario,Cast_testo,Produzione_testo,Critica,Average_rating,Num_votes,Indice_generale,Indice_tipo
0,Age of Samurai: Battle for Japan,NaN,Netflix,NaN,Masayoshi Haneda Masami Kosaka Hideaki Itō Hay...,NaN,43–45 minutes,NaN,1,6,Age of Samurai: Battle for Japan is an America...,NaN,NaN,NaN,7.5,2218.0,0.023967,0.027273
1,Agents of S.H.I.E.L.D.,United States,ABC Studios Marvel Television Mutant Enemy Pro...,Walt Disney Television,Clark Gregg Ming-Na Wen Brett Dalton Chloe Ben...,Joss Whedon Jed Whedon Maurissa Tancharoen,41–44 minutes,Action Drama Science fiction Superhero,7,136 (list of episodes),Marvel's Agents of S.H.I.E.L.D. is an American...,Section: Cast and characters (1):\nClark Gregg...,Section: Production (1):\n\nSubsections (8):\n...,Section: Reception (1):\n\nSubsections (4):\nS...,7.5,201304.0,0.000000,0.019835
2,Alice in Borderland,Japan,Robot Communications Inc.,Netflix,Kento Yamazaki Tao Tsuchiya Yūki Morinaga Keit...,NaN,41–52 minutes,Science fiction[1] Suspense-thriller[2] Drama[3],1,8 (list of episodes),"Alice in Borderland (Japanese: 今際の国のアリス, Hepbu...",Section: Cast and characters (1):\n\nSubsectio...,Section: Production (1):\n\nSubsections (3):\n...,Section: Reception (1):\n\nSubsections (2):\nS...,7.7,18897.0,0.047934,0.097521


### Esporto in file csv

In [23]:
movies.to_csv("film_integrati.csv")

In [24]:
tv_series.to_csv("serie_tv_integrate.csv")

In [25]:
movies.shape

(140, 19)

In [26]:
tv_series.shape

(68, 18)

## Calcolo i 5 titoli più visti

In [33]:
totale = pd.concat([movies[['Titolo', 'Indice_generale', 'Indice_tipo']].copy(), 
                    tv_series[['Titolo', 'Indice_generale', 'Indice_tipo']]].copy(), ignore_index = True)

In [34]:
totale['Popolarita'] = totale['Indice_generale'] + totale['Indice_tipo']

In [36]:
totale.sort_values(by = 'Popolarita', ascending = False).head(30)

,Titolo,Indice_generale,Indice_tipo,Popolarita
147,Bridgerton,0.400826,0.462810,0.863636
198,The Queen's Gambit,0.310744,0.432231,0.742975
169,Lupin,0.281818,0.340496,0.622314
161,Ginny & Georgia,0.259504,0.266942,0.526446
174,New Amsterdam,0.216529,0.247934,0.464463
177,Riverdale,0.207438,0.219008,0.426446
159,Firefly Lane,0.166116,0.183471,0.349587
158,Fate: The Winx Saga,0.147934,0.196694,0.344628
180,Snowpiercer,0.131405,0.208264,0.339669
84,Rose Island,0.125620,0.195041,0.320661


Esporto il dataset finale, con popolarità di film e serie-tv insieme.

In [40]:
totale.to_csv("dataset_popolarita.csv")